## Questions

1. What is the average applying duration of team member?
2. How many ineffective hours are they employing?
3. How many of the job titles are irrelevant? 
4. During which time, most of the working is done by each member?




In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('./applied_jobs-2024-07-27 09_26_39 -0700.csv', low_memory=False)
# df.drop(columns=["Unnamed: 16"], inplace=True)
# df.dropna(inplace=True)
display(df.tail(3))
display(df.shape)

,Job ID,Job Created At,Job Title,Job Location,Job Region,Posted At,Company ID,Job Role,Job Source,Primary Tech Stack,Secondary Tech Stacks,Job Types,Job Created By,Job Applied By,Applied Job ID,Applied At,Pseudo,Vertical
1763,1840019,25/Jul/2024 09:40,"Senior Software Engineer - Frontend,",Remote,Us,2024-07-25 00:00:00 -0700,109975,Dev,Glassdoor,Node.js,JavaScript/TypeScript and React,Full Time,Ans Bin Asif (ID - 2295),Muhammad Ali (ID - 1566),1810935,26/Jul/2024 16:05,Naveed H Zaidi - TechXpark,Ned Hassan - Dev
1764,1839992,25/Jul/2024 09:27,Senior Software Engineer (React),Remote,Us,2024-07-25 00:00:00 -0700,111582,Dev,builtin,Frontend,React,Full Time,Muhammad Tausif ul Hassan (ID - 2159),Muhammad Ali (ID - 1566),1810942,26/Jul/2024 16:07,Naveed H Zaidi - TechXpark,Ned Hassan - Dev
1765,1839899,25/Jul/2024 08:49,"Senior Full-Stack Engineer (NodeJS, React, Typ...",Remote,Us,2024-07-25 00:00:00 -0700,101631,Dev,Glassdoor,Node.js,React and TypeScript,Full Time,Ans Bin Asif (ID - 2295),Muhammad Ali (ID - 1566),1810949,26/Jul/2024 16:11,Naveed H Zaidi - TechXpark,Ned Hassan - Dev


(1766, 18)

In [19]:
# Applying Counts
appliers = df['Job Applied By'].unique()
for applier in appliers:
    print(f"{applier}".ljust(40) + f"=> {df[df['Job Applied By']==applier]['Job ID'].count()}")

Zunaira Afzal (ID - 1579)               => 212
Irtaza Afzal (ID - 2055)                => 268
Ahsan Mehmood (ID - 2016)               => 128
Muhammad Ali (ID - 1566)                => 235
Ali Husnain (ID - 1496)                 => 72
Muhammad Hamza (ID - 1850)              => 264
Kanwar Zahid Sarfraz (ID - 2064)        => 215
Muhammad Kaleem Azhar (ID - 2181)       => 372


In [20]:
# Effective Working in Hours
df['Applied At'] = pd.to_datetime(df['Applied At'])
df['Applied At Date'] = df['Applied At'].dt.strftime('%Y-%m-%d')

# Define start and end time for filtering
start_time = '2024-07-26 05:00'
end_time = '2024-07-26 15:00'
effective_min_diff = 10

df_filtered = df[(df['Applied At'] >= start_time) & (df['Applied At'] <= end_time)]
df_filtered.sort_values(by=['Job Applied By', 'Applied At'], inplace=True)
df_filtered['Time Difference'] = df_filtered.groupby('Job Applied By')['Applied At'].diff()
# display(df_filtered[df_filtered['Job Applied By']=='Ahsan Mehmood (ID - 2016)'].sort_values(by="Applied At"))

effective_times = df_filtered[df_filtered['Time Difference'] <= pd.Timedelta(minutes=effective_min_diff)].groupby(['Applied At Date', 'Job Applied By'])['Time Difference'].sum()
effective_times_df = effective_times.reset_index().pivot(index='Job Applied By', columns='Applied At Date', values='Time Difference')
effective_times_df

C:\Users\Dev\AppData\Local\Temp\ipykernel_14036\2524892336.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Dev\AppData\Local\Temp\ipykernel_14036\2524892336.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Applied At Date,2024-07-26
Job Applied By,
Ahsan Mehmood (ID - 2016),0 days 01:52:00
Ali Husnain (ID - 1496),0 days 01:13:00
Irtaza Afzal (ID - 2055),0 days 01:25:00
Kanwar Zahid Sarfraz (ID - 2064),0 days 00:09:00
Muhammad Ali (ID - 1566),0 days 01:41:00
Muhammad Hamza (ID - 1850),0 days 01:28:00
Muhammad Kaleem Azhar (ID - 2181),0 days 01:24:00
Zunaira Afzal (ID - 1579),0 days 00:55:00


In [21]:

def time_difference_adjustment(x):
    if pd.isna(x):
        return pd.NaT
    elif x <= pd.Timedelta(minutes=effective_min_diff):
        return x
    else:
        pd.Timedelta(minutes=4)
df['Applied At'] = pd.to_datetime(df['Applied At'])
df['Applied At Date'] = df['Applied At'].dt.strftime('%Y-%m-%d')
min_df_date = pd.to_datetime(df['Applied At Date'].min())
max_df_date = pd.to_datetime(df['Applied At Date'].max())
# Date and time picker widgets
start_date = st.sidebar.date_input("Start Date", value=min_df_date)
start_time = st.sidebar.time_input("Start Time", value=datetime.strptime("05:00", "%H:%M").time())
end_date = st.sidebar.date_input("End Date", value=max_df_date)
end_time = st.sidebar.time_input("End Time", value=datetime.strptime("15:00", "%H:%M").time())

# Combine date and time into datetime objects
start_datetime = datetime.combine(start_date, start_time)
end_datetime = datetime.combine(end_date, end_time)
start_time = pd.to_datetime(start_datetime)
end_time = pd.to_datetime(end_datetime)

applier_options = df['Job Applied By'].unique().tolist()
selected_applier = st.multiselect('Select an Applier:', applier_options)
st.header('Effective Working in Hours By Applier')

matches = df['Job Applied By'].isin(selected_applier)
df_filtered = df[matches]

df_filtered = df_filtered[(df_filtered['Applied At'] >= start_time) & (df_filtered['Applied At'] <= end_time)]
df_filtered.sort_values(by=['Job Applied By', 'Applied At'], inplace=True)
# TODO: Make average applying adjustment feature. 
df_filtered['Time Difference'] = df_filtered.groupby('Job Applied By')['Applied At'].diff().apply(time_difference_adjustment)
df_filtered

,Job ID,Job Created At,Job Title,Job Location,Job Region,Posted At,Company ID,Job Role,Job Source,Primary Tech Stack,Secondary Tech Stacks,Job Types,Job Created By,Job Applied By,Applied Job ID,Applied At,Pseudo,Vertical,Applied At Date,Time Difference


In [22]:
appliers = df['Job Applied By'].unique()
data_filtered = df.groupby(['Applied At Date', 'Job Applied By']).count()
data_filtered = data_filtered.reset_index().pivot(index='Job Applied By', columns="Applied At Date", values='Job ID')
data_filtered

Applied At Date,2024-07-22,2024-07-23,2024-07-24,2024-07-25,2024-07-26
Job Applied By,,,,,
Ahsan Mehmood (ID - 2016),51.0,1.0,12.0,20.0,44.0
Ali Husnain (ID - 1496),21.0,15.0,14.0,NaN,22.0
Irtaza Afzal (ID - 2055),72.0,68.0,61.0,NaN,67.0
Kanwar Zahid Sarfraz (ID - 2064),13.0,70.0,70.0,58.0,4.0
Muhammad Ali (ID - 1566),60.0,60.0,50.0,15.0,50.0
Muhammad Hamza (ID - 1850),68.0,61.0,61.0,33.0,41.0
Muhammad Kaleem Azhar (ID - 2181),110.0,30.0,121.0,NaN,111.0
Zunaira Afzal (ID - 1579),61.0,59.0,7.0,35.0,50.0


In [23]:
# Hunt to Apply Ratio
df_filtered = df[df['Job Applied By']==df['Job Created By']].groupby('Job Applied By')['Job ID'].count()
df_filtered

Job Applied By
Ahsan Mehmood (ID - 2016)             31
Ali Husnain (ID - 1496)               71
Irtaza Afzal (ID - 2055)              53
Kanwar Zahid Sarfraz (ID - 2064)      78
Muhammad Ali (ID - 1566)              33
Muhammad Hamza (ID - 1850)            55
Muhammad Kaleem Azhar (ID - 2181)    103
Zunaira Afzal (ID - 1579)              7
Name: Job ID, dtype: int64

In [24]:
# List the job titles applied by each applier

applier_job_titles = df.groupby('Job Applied By')['Job Title'].apply(list).reset_index()

for index, row in applier_job_titles.iterrows():
    print(f"{row['Job Applied By']}: {' |'.join(row['Job Title'])}")


Ahsan Mehmood (ID - 2016): Sr. Frontend Engineer (Design System) |Lead Software Engineer, |Node.js Developer, |WordPress/Wix/SquareSpace Web Developer |Sr Software Engineer |Lead React Engineers |Sr Front End Software Engineer - US Remote |Senior Software Engineer, |Staff Engineer |Senior Full Stack Engineer, |Full-Stack Engineer |Senior Back-end Node JS Developer with AWS (Remote) |AWS AppSync / TypeScript Developer Lead |Crypto Senior Full Stack Developer (Jakarta - Remote) |Full Stack Developer |FULL STACK SOFTWARE DEVELOPER |Staff Backend Developer |Full Stack Software Engineer |Lead Ruby-on-Rails Engineer |Software Engineerr |SENIOR SOFTWARE ENGINEERR |Full Stack Engineer |Middle Fullstack Developer (Node.js/React.js) |Senior Software Engineer |Software Engineer - Growth |Senior Fullstack Engineer, Core |Senior Software Engineer, Front End |Senior Frontend Developer |Senior Software Engineer |Full Stack Software Engineer |Software Engineer II (Full Stack) |Software Engineer - Full

In [25]:
# Majorit Work Done?
import matplotlib.pyplot as plt

df['Applied At'] = pd.to_datetime(df['Applied At'])

# Extract the hour from 'Applied At'
df['Hour'] = df['Applied At'].dt.hour

# Count the number of job applications for each applier per hour
hourly_counts = df.groupby(['Job Applied By', 'Hour']).size().unstack(fill_value=0)

# Plotting the data
fig, ax = plt.subplots(figsize=(15, 8))

for applier in hourly_counts.index:
    ax.plot(hourly_counts.columns, hourly_counts.loc[applier], marker='o', label=applier)

ax.set_title('Hourly Work Done by Each Applier')
ax.set_xlabel('Hour of the Day')
ax.set_ylabel('Number of Applications')
ax.legend(title='Applier')
ax.grid(True)
plt.xticks(range(24))  # Ensures all 24 hours are shown on the x-axis
plt.show()


C:\Users\Dev\AppData\Local\Temp\ipykernel_14036\1198780610.py:24: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



In [26]:
import plotly.express as px

# Convert 'Applied At' to datetime if not already done
df['Applied At'] = pd.to_datetime(df['Applied At'])

# Extract the hour from 'Applied At'
df['Hour'] = df['Applied At'].dt.hour

# Count the number of job applications for each applier per hour
hourly_counts = df.groupby(['Job Applied By', 'Hour']).size().reset_index(name='Count')

# Create an interactive plot using Plotly
fig = px.line(hourly_counts, x='Hour', y='Count', color='Job Applied By', markers=True,
              title='Hourly Work Done by Each Applier',
              labels={'Hour': 'Hour of the Day', 'Count': 'Number of Applications', 'Job Applied By': 'Applier'})

fig.update_traces(mode='lines+markers')

fig.show()


In [27]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime

st.set_page_config(layout="wide")
st.title('Job Application Analysis')
effective_min_diff = 9

def time_difference_adjustment(x):
    if pd.isna(x):
        return pd.NaT
    elif x <= pd.Timedelta(minutes=effective_min_diff):
        return x
    else:
        pd.Timedelta(minutes=4)

# Upload CSV file
uploaded_file = st.file_uploader("Choose a file")
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    
    df['Applied At'] = pd.to_datetime(df['Applied At'])
    df['Applied At Date'] = df['Applied At'].dt.strftime('%Y-%m-%d')
    min_df_date = pd.to_datetime(df['Applied At Date'].min())
    max_df_date = pd.to_datetime(df['Applied At Date'].max())
    # Date and time picker widgets
    start_date = st.sidebar.date_input("Start Date", value=min_df_date)
    start_time = st.sidebar.time_input("Start Time", value=datetime.strptime("05:00", "%H:%M").time())
    end_date = st.sidebar.date_input("End Date", value=max_df_date)
    end_time = st.sidebar.time_input("End Time", value=datetime.strptime("15:00", "%H:%M").time())

    # Combine date and time into datetime objects
    start_datetime = datetime.combine(start_date, start_time)
    end_datetime = datetime.combine(end_date, end_time)
    start_time = pd.to_datetime(start_datetime)
    end_time = pd.to_datetime(end_datetime)

    applier_options = df['Job Applied By'].unique().tolist()
    selected_applier = st.multiselect('Select an Applier:', applier_options)
    st.header('Effective Working in Hours By Applier')

    matches = df['Job Applied By'].isin(selected_applier)
    df_filtered = df[matches]

    df_filtered = df_filtered[(df_filtered['Applied At'] >= start_time) & (df_filtered['Applied At'] <= end_time)]
    df_filtered.sort_values(by=['Job Applied By', 'Applied At'], inplace=True)
    # TODO: Make average applying adjustment feature. 
    df_filtered['Time Difference'] = df_filtered.groupby('Job Applied By')['Applied At'].diff().apply(time_difference_adjustment)
    